# Randomized Benchmarking


*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*


## Outline

**Note: The number of points in the Quantum Hub account and time spent to run this tutorial program will vary based on the parameters users input. Users usually need about half-hour and 100 points to obtain relatively reliable results. If you want to get more points, please contact us on [Quantum Hub](https://quantum-hub.baidu.com). First, you should log into [Quantum Hub](https://quantum-hub.baidu.com), then enter the "Feedback" page, choose "Get Credit Point", and input the necessary information. Submit your feedback and wait for a reply.**

There are generally two ways in experiments for characterizing the performance of a quantum computer in a superconducting platform: Quantum Process Tomography(QPT) and Randomized Benchmarking(RB) \[1\]. QPT can completely characterize a gate, decomposing a process into Pauli or Kraus operators, but improving gates by QPT is complicated and resource-consuming. Also, State Preparation And Measurement (SPAM) errors can be confused with process errors. However, RB is a concept of using randomization methods for benchmarking quantum gates. It is a scalable and SPAM robust method for benchmarking the full set of gates by a single parameter using randomization techniques. So it is useful to use RB for a relatively simple and at least SPAM-robust benchmark, especially when the number of qubit increases.

In this tutorial, we will implement RB on one of the qubits in our noisy simulator defined in advance to characterize the average error rate on a Hadamard gate. The outline is as follow:

- Introduction
- Preparation
- Define the hardware to benchmark
- Running RB
- Summary

## Introduction


**Basic RB**

We usually use pure Clifford gates to get a sequence fidelity decay curve by randomly choose $m$ Clifford gates followed by an inverse Clifford gate to make sure that the effect of all $m+1$ Clifford gates is an identity operation $\mathcal{I}$ and then apply them to a qubit:

![basicRB](figures/basicRB.png)

The above circuit shows how our basic randomized benchmarking works. $C_{i}$ represents the $i_{th} (i = 1, 2, 3, \dots, m)$ Clifford gates we randomly choose. Ideally, assume our qubit's initial state is $|\psi\rangle$, and there is no noise in this RB procedure, we will get the final state being $|\psi\rangle$ for $100\%$ probability, and we use the probability of the final state still being $|\psi\rangle$ as a measure of our RB sequence fidelity \[2\]. However, in reality, this sequence fidelity $\mathcal{F}$ decays when the number of gates increases because of the accumulation of noise. If the noise distribution is time- and gate-independent, such decay can be described by a zeroth fitting function:

$$
\mathcal{F}^{(0)}=Ap_{\rm basic}^m+B,
$$

where $m$ is the number of Cliffords we apply. For more detailed basic knowledge and related theories about randomized benchmarking, users can refer to \[3\].


The advantage of RB is that the SPAM error is included in the parameters $A$ and $B$ without affecting the rate of decay parameter $p$. More specifically, if we decompose the initial density operator $\rho$ and measurement operator $\hat{E}$ into Pauli basis $\hat{P}_i$: 

$$
\rho=\sum_jx_j\hat{P}_i/d,
$$

$$
\hat{E}=\sum_j\tilde{e}_j\hat{P}_j,
$$

then $A = \sum_{j\neq 0}\tilde{e}_jx_j$, and $B = \tilde{e}_0$ where $d\equiv{2^n}$ and $n$ is the number of qubits. Once we successfully get $p_{basic}$ by fitting the curve, we can obtain the average EPC(Error-rate Per Clifford) immediately:

$$
{\rm EPC}=\frac{(1-p_{\rm basic})(d-1)}{d}.
$$


**Interleaved RB**

Interleaved randomized benchmarking is used to get the average error rate of a particular quantum gate. After we obtained a reference sequence fidelity decay curve by first implementing the basic RB, we can then apply the so-called interleaved randomized benchmarking \[2\] to benchmark the performance of the Hadamard gate. We interleaved our aimed gate behind every time we implement a random Clifford gate and then make the gate sequence equals to the identity gate by an inverse Clifford gate. The interleaved randomized benchmarking sequence shown below takes the Hadamard gate(H gate) being the target gate as an example: :

![interleavedRB](figures/interleavedRB.png)

And we can as well get a sequence fidelity decay curve that is similarily described by:

$$
\mathcal{F}^{(0)\prime}=A^{\prime}p_{\rm gate}^m+B^{\prime}.
$$

Consequently, we calculate the average error rate on the aimed gate EPG(Error-rate Per Gate) by subtracting the reference curve away:

$$
r_{\rm gate}=\frac{(1-p_{\rm gate}/p_{\rm ref})(d-1)}{d}.
$$

As we can see, this gate error $r$ characterizes the gate's performance we want to benchmark since it is derived from a set of experiments' data.

Next, we will use Quanlse to implement the RB experiment to benchmark a Hadamard gate. As mentioned above, since we are going to benchmark a specific gate, we need to use both basic RB and interleaved RB. It is worth mentioning that in the up-to-date version of Quanlse, we only support the single-qubit Randomized Benchmarking with Clifford basis. 

## Preparation

First, we need to import the following necessary packages and get a token in order to use the cloud service:

In [ ]:
# Import the necessary packages
from Quanlse.Utils.RandomizedBenchmarking import RB
from Quanlse.Utils.Functions import basis, tensor
from Quanlse.QOperation import FixedGate
from Quanlse.Simulator import PulseModel
from Quanlse.Scheduler.Superconduct import SchedulerSuperconduct
from Quanlse.Scheduler.Superconduct.RBPulseGenerator import SingleQubitCliffordPulseGenerator
from Quanlse import Define

from math import pi
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt

# Import Define class and set the token
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ''

## Define the Hardware to Benchmark

Then we need to construct a virtual quantum hardware - a noisy simulator and decide which of the qubit and gates we want to benchmark.

Quanlse supports user-defined multi-qubit noisy simulator, for more detail, users can refer to [multi-qubit noisy simulator](https://quanlse.baidu.com/#/doc/tutorial-multi-qubit-noisy-simulator). Here, we build up a two-qubit system using Quanlse. Each qubit takes three energy levels into consideration. Then, we want to benchmark the performance of the Hadamard gate implementation on the first qubit of this virtual hardware: 

In [ ]:
# Define the basic parameters of the simulator
sysLevel = 3  # The number of energy levels of each qubit
qubitNum = 2  # The number of qubits simulator has

# Qubit frequency & anharmonicity
wq0 = 5.282 * (2 * pi)  # The frequency for qubit 0, in 2 pi GHz
wq1 = 5.248 * (2 * pi)  # The frequency for qubit 1, in 2 pi GHz
anharm0 = - 0.33081 * (2 * pi)  # The anharmonicity for qubit 0, in 2 pi GHz
anharm1 = - 0.33043 * (2 * pi)  # The anharmonicity for qubit 1, in 2 pi GHz
qubitFreq = {0: wq0, 1: wq1}
qubitAnharm = {0: anharm0, 1: anharm1}

# Coupling map between qubits
g01 = 0.002 * (2 * pi)  # The coupling strength of the interaction between qubit 0 and qubit 1, in 2 pi GHz
couplingMap = {(0, 1): g01}

# Taking T1 & T2 dissipation into consideration, in the unit of nanosecond
t1List = {0: 50310, 1: 62200}
t2List = {0: 13630, 1: 24280}

# Sampling time
dt = 1.  

# Build a virtual QPU
model = PulseModel(subSysNum=qubitNum,
                   sysLevel=sysLevel,
                   couplingMap=couplingMap,
                   qubitFreq=qubitFreq,
                   dt=dt,
                   qubitAnharm=qubitAnharm,
                   T1=t1List, T2=t2List,
                   ampSigma=0.0001)
ham = model.createQHamiltonian()

# The initial state of this simulator
initialState = tensor(basis(3, 0), basis(3, 0))

# Decide the qubit we want to benchmark
targetQubitNum = 0
hamTarget = ham.subSystem(targetQubitNum)

# Decide one specific gate we want to benchmark
targetGate = FixedGate.H

And now we are ready to execute the RB experiment on our chosen qubit.

Since we will obtain plenty of pulses when appling RB experiment, we have to use `SchedulerSuperconduct()` to efficiently schedule those pulses:

In [ ]:
sche = SchedulerSuperconduct(dt=dt, ham=hamTarget, pulseGenerator=SingleQubitCliffordPulseGenerator(hamTarget))

## Running RB

We can obtain a reference curve by doing basic RB calling ``RB`` function which takes some input into consideration: the system ``model`` we have and its initial state ``initialState``, the index ``targetQubitNum`` of the qubit in this system we want to benchmark, the list of Clifford gate number `size` and the number of sequences ``width`` every element in ``size`` has, the scheduler `sche` we initiate before, the sampling time `dt`. We also have to decide whether to use the interleaved RB method. If `interleaved=True`, which means we now implement interleaved RB, then the parameter `targetGate` representing the gate we want to benchmark must be included. In this tutorial, we take qubit's $T_1$ and $T_2$ into consideration, which means that we simulate the RB experiments within an open system, so we set `isOpen=True`:

In [ ]:
# Create a list to store the outcome
sizeSequenceFidelityBasic = []
sizeSequenceFidelityInterleaved = []

# Core parameters of an RB experiment
size = [1, 10, 20, 50, 75, 100, 125, 150, 175, 200]
width = 5

# Start RB experiment. First get a basicRB curve used for reference. Then implement the interleavedRB to benchmark our Hadamard gate
for i in size:
    widthSequenceFidelityBasic = RB(model=model, targetQubitNum=targetQubitNum, initialState=initialState, size=i, width=width, sche=sche,
                                    dt=dt, interleaved=False, isOpen=True)
    sizeSequenceFidelityBasic.append(widthSequenceFidelityBasic)
print(sizeSequenceFidelityBasic)
    
for j in size:
    widthSequenceFidelityInterleaved = RB(model=model, targetQubitNum=targetQubitNum, initialState=initialState, size=j, width=width,
                                          targetGate=targetGate, sche=sche, dt=dt, interleaved=True, isOpen=True)
    sizeSequenceFidelityInterleaved.append(widthSequenceFidelityInterleaved)
print(sizeSequenceFidelityInterleaved)

After we successfully get plenty of sequence fidelity data that these two RB method produced, we then fit the curve to obtain both the average EPC and the average EPG:

In [ ]:
# Define the fitting function
def fit(x, a, p, b):
    """
    Define the fitting curve
    """
    return a * (p ** x) + b

# Define the function of calculating the EPG(Error-rate Per Gate) with p_{gate} and p_{ref}
def targetGateErrorRate(pGate, pRef, dimension):
    """
    Calculate the specific gate error rate
    """
    return ((1 - (pGate / pRef)) * (dimension - 1)) / dimension


# Get the EPC(Error-rate Per Clifford) and p_{ref}
fitparaBasic, fitcovBasic = curve_fit(fit, size, sizeSequenceFidelityBasic, p0=[0.5, 1, 0.5], maxfev=500000,
                                      bounds=[0, 1])
pfitBasic = fitparaBasic[1]
rClifford = (1 - pfitBasic) / 2
print('EPC =', rClifford)

# Get the parameter p_{gate}
fitparaInterleaved, fitcovInterleaved = curve_fit(fit, size, sizeSequenceFidelityInterleaved,
                                                  p0=[fitparaBasic[0], 1, fitparaBasic[2]], maxfev=500000,
                                                  bounds=[0, 1])
pfitInterleaved = fitparaInterleaved[1]
yfitBasic = fitparaBasic[0] * (pfitBasic ** size) + fitparaBasic[2]
yfitInterleaved = fitparaInterleaved[0] * (pfitInterleaved ** size) + fitparaInterleaved[2]
EPG = targetGateErrorRate(pfitInterleaved, pfitBasic, dimension=2)
print('EPG =', EPG)

And plot the curve to obtain the result of the decay curve of sequence fidelity for our RB experiment:

In [ ]:
# Plot the decay curve of our RB experiment
plt.figure(figsize=(18, 6), dpi=80)
plt.figure(1)
ax1 = plt.subplot(121)
ax1.plot(size, sizeSequenceFidelityBasic, '.b', label='experiment simulation data')
ax1.plot(size, yfitBasic, 'r', label='fitting curve')
plt.xlabel('$m$')
plt.ylabel('Sequence Fidelity')
plt.title('basic RB using Quanlse')
plt.legend()
ax2 = plt.subplot(122)
ax2.plot(size, sizeSequenceFidelityInterleaved, '.b', label='experiment simulation data')
ax2.plot(size, yfitInterleaved, 'r', label='fitting curve')
plt.xlabel('$m$')
plt.ylabel('Sequence Fidelity')
plt.title('interleaved RB using Quanlse')
plt.legend()
plt.show()

Here, $m$ represents the number of Clifford gates we applied. The curve shown in the figure reflects the phenomenon that the accumulated noise as the number of gates (the number of pulses) increases causes the sequence fidelity to decay exponentially. It can be seen that with this scheme, we can automatically generate high-precision pulses that are adapted to the gate operation of the target quantum hardware, perform pulse scheduling when the number of pulses increases significantly, and further conduct randomized benchmarking experiments on the quantum hardware.

## Summary

This tutorial describes how to implement RB experiments to benchmark the performance of a specific gate on one of the qubits in our noisy simulator. Users can try the different parameter values for better curve fitting, and further exploring the variant method of the Randomized Benchmarking.

## Reference
\[1\] [Kelly, Julian, et al. "Optimal quantum control using randomized benchmarking." Physical review letters 112.24 (2014): 240504.](https://doi.org/10.1103/PhysRevLett.112.240504)

\[2\] [Magesan, Easwar, et al. "Efficient measurement of quantum gate error by interleaved randomized benchmarking." Physical review letters 109.8 (2012): 080505.](https://doi.org/10.1103/PhysRevLett.109.080505)

\[3\][Magesan, Easwar, Jay M. Gambetta, and Joseph Emerson. "Scalable and robust randomized benchmarking of quantum processes." Physical review letters 106.18 (2011): 180504.](https://doi.org/10.1103/PhysRevLett.106.180504)